# Part I

In [ ]:
!pip install transformers[torch] datasets evaluate seqeval

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_in

## Load data

In [ ]:
from datasets import load_dataset

In [ ]:
wnut = load_dataset("wnut_17")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
# wnut["train"][0]

## Preprocess

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from']

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
# tokenized_wnut
# tokenized_wnut["train"]
tokenized_wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  -100,
  -100,
  0,
  0,
  -100,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

In [ ]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

## Train

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
)
model

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
example = tokenized_wnut["train"][0]
# x = torch.Tensor(example["input_ids"]).unsqueeze(0).type(torch.int).to(device)
# x.shape
# pred = model(x)
# pred.logits.shape

In [ ]:
import torch

device = torch.device('cuda')
model = model.to(device)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # параметр, который нужно будет дергать
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Inference

In [ ]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [ ]:
from transformers import pipeline

classifier = pipeline("ner", model="my_awesome_wnut_model/checkpoint-426")
classifier(text)

# Part II

In [ ]:
!pip install transformers[torch] datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=5f55731bca3e88d68fbcbc24a84dce8161da13370d9ee302d5eb3c5fb3b6cffe
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import evaluate
seqeval = evaluate.load("seqeval")
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch

device = torch.device('cuda')
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import pipeline

from parasite_context_dataset import parasite_context_dataset
from normal_context_dataset import normal_context_dataset

from nltk.tokenize import WordPunctTokenizer

from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

from transformers import DataCollatorForTokenClassification

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
bad_words = [
  "вроде",
  "типа",
  "как бы",
  "это самое",
  "короче",
  "на самом деле",
  "собственно",
  "таким образом",
  "как говорится",
  "как его",
  "вообще-то",
  "просто",
  "в общем",
  "типа",
  "в целом",
  "то есть",
  "как бы",
  "блин",
  "пипец",
  "капец",
  "офигеть",
  "нихрена",
  "нифига себе",
  "ах",
  "ох",
  "эх",
  "ух",
  "ай",
  "ой",
  "ну",
  "ага",
  "эй",
  "фу",
  "брр",
  "хм",
  "пф",
  "тьфу",
  "ура",
  "увы",
  "ишь",
  "эге",
  "ба",
  "нате"
]

## Load data

In [ ]:
raw_dataset_dialogue = load_dataset("Den4ikAI/russian_dialogues", split='train[0:10000]')

README.md:   0%|          | 0.00/951 [00:00<?, ?B/s]

dataset.jsonl:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2477321 [00:00<?, ? examples/s]

In [ ]:
# raw_dataset_dialogue['question'][:10]

## Preprocess

In [ ]:
wptokenizer = WordPunctTokenizer()

tokenized_raw_dataset_dialogue = [wptokenizer.tokenize(line.lower()) for line in raw_dataset_dialogue["question"] + raw_dataset_dialogue["answer"] if line is not None]

In [ ]:
parasite_context_tokenized_dataset = [wptokenizer.tokenize(line.lower()) for word in parasite_context_dataset for line in parasite_context_dataset[word]] # токенизация list[str] -> list[list[str]]
normal_context_tokenized_dataset = [[wptokenizer.tokenize(line.lower()), word] for word in normal_context_dataset for line in normal_context_dataset[word]]

In [ ]:
parasite_context_tokenized_dataset[:10]

[['я', 'вроде', 'уже', 'видел', 'этот', 'фильм'],
 ['она', 'вроде', 'была', 'на', 'вечеринке', 'вчера'],
 ['вроде', 'мы', 'договорились', 'встретиться', 'в', '7'],
 ['это', 'вроде', 'не', 'так', 'сложно', 'как', 'кажется'],
 ['он', 'вроде', 'согласился', 'помочь'],
 ['она', 'вроде', 'говорила', 'об', 'этом'],
 ['вроде', 'это', 'был', 'его', 'план'],
 ['он', 'вроде', 'учился', 'в', 'этом', 'университете'],
 ['вроде', 'это', 'не', 'моя', 'ошибка'],
 ['она', 'вроде', 'была', 'здесь', 'вчера']]

In [ ]:
normal_context_tokenized_dataset[:10]

[[['я', 'объявил', 'переменную', 'целочисленного', 'типа'], 'типа'],
 [['эта', 'программа', 'использует', 'объект', 'типа', 'строка'], 'типа'],
 [['в', 'меню', 'есть', 'блюда', 'типа', 'пасты', 'и', 'пиццы'], 'типа'],
 [['он', 'работает', 'в', 'компании', 'типа', 'стартапа'], 'типа'],
 [['в', 'его', 'коллекции', 'есть', 'модели', 'типа', 'автомобилей'], 'типа'],
 [['мы', 'изучаем', 'функции', 'типа', 'преобразования', 'данных'], 'типа'],
 [['этот', 'фильм', 'в', 'жанре', 'типа', 'драма', 'и', 'комедия'], 'типа'],
 [['нам', 'нужны', 'данные', 'типа', 'числовые', 'для', 'анализа'], 'типа'],
 [['проект', 'требует', 'подхода', 'типа', 'анализа', 'рынка'], 'типа'],
 [['эта', 'ткань', 'имеет', 'свойства', 'типа', 'водоотталкивающей'], 'типа']]

In [ ]:
import pandas as pd

ner_dataset_dialogue = []
for sentence in tokenized_raw_dataset_dialogue:
    if sentence is None:
        continue
    ner_tags = []
    if sentence[0] in bad_words:
        ner_tags.append(1)
    else:
        ner_tags.append(0)
    for index, word in enumerate(sentence[1:]):
        ner_tag = 0
        if word in bad_words:
            ner_tag = 1
        if ' '.join([sentence[index], word]) in bad_words:
            ner_tags[index] = 1
            ner_tag = 1
        ner_tags.append(ner_tag)
    ner_dataset_dialogue.append([sentence, ner_tags])

# ner_dataset_dialogue

In [ ]:
# разметка ner_tags raw_dataset: list[list[str]] -> ner_dataset: list[list[str], list[int]]

ner_dataset_parasites = []
for sentence in parasite_context_tokenized_dataset:
    if sentence is None:
        continue
    ner_tags = []
    if sentence[0] in bad_words:
        ner_tags.append(1)
    else:
        ner_tags.append(0)
    for index, word in enumerate(sentence[1:]):
        ner_tag = 0
        if word in bad_words:
            ner_tag = 1
        if ' '.join([sentence[index], word]) in bad_words:
            ner_tags[index] = 1
            ner_tag = 1
        ner_tags.append(ner_tag)
    ner_dataset_parasites.append([sentence, ner_tags])

# ner_dataset_parasites

In [ ]:
ner_dataset_normal = []
for sentence in normal_context_tokenized_dataset:
    if sentence[0] is None:
        continue

    ner_tags = []
    if sentence[0][0] == sentence[1]:
        ner_tags.append(0)
    elif sentence[0][0] in bad_words:
        ner_tags.append(1)
    else:
        ner_tags.append(0)
    for index, word in enumerate(sentence[0][1:]):
        ner_tag = 0
        if ' '.join([sentence[0][index], word]) == sentence[1]:
            ner_tags[index] = 0
            ner_tag = 0
        elif str(word) == str(sentence[1]):
            ner_tag = 0
        elif ' '.join([sentence[0][index], word]) == sentence[1]:
            ner_tags[index] = 0
            ner_tag = 0
        elif word in bad_words:
            ner_tag = 1
        else:
            ner_tag = 0
        ner_tags.append(ner_tag)
    ner_dataset_normal.append([sentence[0], ner_tags])


# ner_dataset_normal


In [ ]:
ner_dataset_parasites[:10]

[[['я', 'вроде', 'уже', 'видел', 'этот', 'фильм'], [0, 1, 0, 0, 0, 0]],
 [['она', 'вроде', 'была', 'на', 'вечеринке', 'вчера'], [0, 1, 0, 0, 0, 0]],
 [['вроде', 'мы', 'договорились', 'встретиться', 'в', '7'],
  [1, 0, 0, 0, 0, 0]],
 [['это', 'вроде', 'не', 'так', 'сложно', 'как', 'кажется'],
  [0, 1, 0, 0, 0, 0, 0]],
 [['он', 'вроде', 'согласился', 'помочь'], [0, 1, 0, 0]],
 [['она', 'вроде', 'говорила', 'об', 'этом'], [0, 1, 0, 0, 0]],
 [['вроде', 'это', 'был', 'его', 'план'], [1, 0, 0, 0, 0]],
 [['он', 'вроде', 'учился', 'в', 'этом', 'университете'], [0, 1, 0, 0, 0, 0]],
 [['вроде', 'это', 'не', 'моя', 'ошибка'], [1, 0, 0, 0, 0]],
 [['она', 'вроде', 'была', 'здесь', 'вчера'], [0, 1, 0, 0, 0]]]

In [ ]:
ner_dataset_parasites

[[['я', 'вроде', 'уже', 'видел', 'этот', 'фильм'], [0, 1, 0, 0, 0, 0]],
 [['она', 'вроде', 'была', 'на', 'вечеринке', 'вчера'], [0, 1, 0, 0, 0, 0]],
 [['вроде', 'мы', 'договорились', 'встретиться', 'в', '7'],
  [1, 0, 0, 0, 0, 0]],
 [['это', 'вроде', 'не', 'так', 'сложно', 'как', 'кажется'],
  [0, 1, 0, 0, 0, 0, 0]],
 [['он', 'вроде', 'согласился', 'помочь'], [0, 1, 0, 0]],
 [['она', 'вроде', 'говорила', 'об', 'этом'], [0, 1, 0, 0, 0]],
 [['вроде', 'это', 'был', 'его', 'план'], [1, 0, 0, 0, 0]],
 [['он', 'вроде', 'учился', 'в', 'этом', 'университете'], [0, 1, 0, 0, 0, 0]],
 [['вроде', 'это', 'не', 'моя', 'ошибка'], [1, 0, 0, 0, 0]],
 [['она', 'вроде', 'была', 'здесь', 'вчера'], [0, 1, 0, 0, 0]],
 [['он', 'вроде', 'сказал', 'что', 'придет'], [0, 1, 0, 0, 0]],
 [['вроде', 'всё', 'готово', 'к', 'началу'], [1, 0, 0, 0, 0]],
 [['она', 'вроде', 'знала', 'об', 'этом'], [0, 1, 0, 0, 0]],
 [['он', 'вроде', 'уже', 'отправил', 'письмо'], [0, 1, 0, 0, 0]],
 [['вроде', 'это', 'правильно'], [1, 0, 0

In [ ]:
# concatenate
ner_dataset_total = ner_dataset_normal + ner_dataset_parasites + ner_dataset_dialogue
# ner_dataset_total = ner_dataset_normal + ner_dataset_parasites
print(len(ner_dataset_total))

21508


In [ ]:
print(ner_dataset_parasites[0])
print(ner_dataset_normal[0])
print(ner_dataset_dialogue[0])

[['я', 'вроде', 'уже', 'видел', 'этот', 'фильм'], [0, 1, 0, 0, 0, 0]]
[['я', 'объявил', 'переменную', 'целочисленного', 'типа'], [0, 0, 0, 0, 0]]
[['как', 'дела', '?'], [0, 0, 0]]


In [ ]:
ner_dataframe_total = pd.DataFrame(ner_dataset_total).sample(frac=1, random_state=42).reset_index() # из типа списка списков в pandas формат
ner_dataframe_total.columns = ["id", "tokens", "ner_tags"]

In [ ]:
ner_dataframe_total.head()

,id,tokens,ner_tags
0,423,"[он, короче, не, смог, прийти]","[0, 1, 0, 0, 0]"
1,3108,"[женщина, за, рулем, мотоцикла, ., почему, муж...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,5719,"[а, что, для, вас, счастье, ?]","[0, 0, 0, 0, 0, 0]"
3,20574,"[если, своя, то, секс, больше, дай, ей, и, вни...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,7547,"[как, научиться, копить, деньги, ?]","[0, 0, 0, 0, 0]"


In [ ]:
ner_dataframe_train, ner_dataframe_test = train_test_split(ner_dataframe_total, test_size=0.01, random_state=42)

In [ ]:
ner_dataset_hf_train = Dataset.from_pandas(ner_dataframe_train)
ner_dataset_hf_test = Dataset.from_pandas(ner_dataframe_test)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, max_length=512, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_ner_dataset_hf_train = ner_dataset_hf_train.map(tokenize_and_align_labels, batched=True)
tokenized_ner_dataset_hf_test = ner_dataset_hf_test.map(tokenize_and_align_labels, batched=True)

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Map:   0%|          | 0/21292 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

In [ ]:
tokenized_ner_dataset_hf_train[0]

{'id': 3110,
 'tokens': ['спать',
  'хочется',
  ',',
  'но',
  'проспать',
  'весь',
  'день',
  'тоже',
  'не',
  'хочется',
  '.',
  'что',
  'делать',
  '?'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 '__index_level_0__': 13201,
 'input_ids': [101,
  8588,
  6508,
  121,
  750,
  3763,
  8578,
  2252,
  1336,
  1553,
  672,
  6508,
  126,
  693,
  2555,
  161,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

## Train

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "ai-forever/ruBert-base", num_labels=2, id2label={0:'0', 1:'1'}, label2id={'0':0, '1':1})

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to(device)

In [ ]:
for param in model.bert.parameters():
    param.requires_grad = False

In [ ]:
for param in model.parameters(): param.data = param.data.contiguous()

In [ ]:
wnut = load_dataset("wnut_17")
example = wnut["train"][0]
label_list = wnut["train"].features[f"ner_tags"].feature.names


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

wnut_17.py:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

The repository for wnut_17 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wnut_17.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# output_dir="rubert_tiny2_dialoge2gpt_new_data",

# TODO training
training_args = TrainingArguments(
    output_dir="recognize_words-parasites_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=13, #дёргаем
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ner_dataset_hf_train,
    eval_dataset=tokenized_ner_dataset_hf_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-30-f5b14150461b>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.092400,0.066238,0.000000,0.000000,0.000000,0.988953
2,0.060400,0.052108,0.000000,0.000000,0.000000,0.988953
3,0.055100,0.043521,0.000000,0.000000,0.000000,0.988953
4,0.044300,0.038311,1.000000,0.095238,0.173913,0.990005
5,0.039600,0.034971,1.000000,0.142857,0.250000,0.990531
6,0.037800,0.032707,1.000000,0.142857,0.250000,0.990531
7,0.035600,0.031081,1.000000,0.238095,0.384615,0.991583
8,0.033200,0.029912,1.000000,0.238095,0.384615,0.991583
9,0.033700,0.029046,1.000000,0.238095,0.384615,0.991583
10,0.032700,0.028440,1.000000,0.285714,0.444444,0.992109


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=17303, training_loss=0.04920374470102107, metrics={'train_runtime': 760.7438, 'train_samples_per_second': 363.849, 'train_steps_per_second': 22.745, 'total_flos': 5113714301844624.0, 'train_loss': 0.04920374470102107, 'epoch': 13.0})

## Inference

In [ ]:
from transformers import pipeline

# parasites
# text = "Ба, какие люди"
# text = "Я, вроде, уже видел этот фильм"
# text = "Это, типа, новое приложение для тренировок."
# text = "Это, типу, должно помочь."
# text = "Это, типу, должно помочь."
# text = "Это, типу, должно помочь."
# text = "Это, типу, должно помочь."
# text = "Это, типу, должно помочь."
# text = "Это, типу, должно помочь."
# text = "Короче, это было классное шоу"

# normal
# text = "Этот путь короче, чем другой"

# by ourself
# text = "мы выбрали длинную дорогу, а могли бы прийти другой дорогой короче."

# text = "Москва - это слишком просто для меня"
# text = "Блин, это было очень жестоко"
# text = "ребята, ну у вас и погода, капец"

text1 = "Короче, это было классное шоу"
text2 = "мы выбрали длинную дорогу, а могли бы прийти другой дорогой короче."
classifier = pipeline("ner", model="recognize_words-parasites_model/checkpoint-17303", device = 0)
result1 = classifier(text1)
result2 = classifier(text2)

Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
for el in result1 + result2:
    if (el["entity"] == '1'):
        print(f"Контекст мусора: {el}")
    else:
      print(f"Нормальный контекст: {el}")

Контекст мусора: {'entity': '1', 'score': 0.72390604, 'index': 1, 'word': 'короче', 'start': 0, 'end': 6}
Нормальный контекст: {'entity': '0', 'score': 0.9882256, 'index': 2, 'word': ',', 'start': 6, 'end': 7}
Нормальный контекст: {'entity': '0', 'score': 0.9894103, 'index': 3, 'word': 'это', 'start': 8, 'end': 11}
Нормальный контекст: {'entity': '0', 'score': 0.977015, 'index': 4, 'word': 'было', 'start': 12, 'end': 16}
Нормальный контекст: {'entity': '0', 'score': 0.9124935, 'index': 5, 'word': 'классно', 'start': 17, 'end': 24}
Нормальный контекст: {'entity': '0', 'score': 0.99705386, 'index': 6, 'word': '##е', 'start': 24, 'end': 25}
Нормальный контекст: {'entity': '0', 'score': 0.99054015, 'index': 7, 'word': 'шоу', 'start': 26, 'end': 29}
Нормальный контекст: {'entity': '0', 'score': 0.9926313, 'index': 1, 'word': 'мы', 'start': 0, 'end': 2}
Нормальный контекст: {'entity': '0', 'score': 0.9969886, 'index': 2, 'word': 'выбрали', 'start': 3, 'end': 10}
Нормальный контекст: {'entity

In [ ]:
result